In [2]:
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize

In [11]:
def extractFeatures():
    artistTags = np.zeros((18746, 12648))
    file = open("user_taggedartists.dat").readlines()
    for line in file[1:]:
        data = line.split('\t')
        artistTags[int(data[1])][int(data[2])] += 1
    artistTags = normalize(sparse.csr_matrix(artistTags))

    userArtists = np.zeros((2101, 18746))
    userGenres = np.zeros((2101, 12648))
    userExp = np.zeros(2101)
    file = open("user_artists.dat").readlines()
    for line in file[1:]:
        data = line.split('\t')
        userGenres[int(data[0])] += artistTags[int(data[1])]*int(data[2])
        userArtists[int(data[0])][int(data[1])] += [int(data[2])]
        userExp[int(data[0])] += 1
    userGenres = normalize(userGenres)
    return userGenres, userExp, userArtists

In [13]:
userGenres, userExp, userArtists = extractFeatures()

In [46]:
class modifiedEuclideanDistRecommender():
    def __init__(self, k, genres, exp, artists):
        self.k = k
        self.exp = exp
        self.genres = genres
        self.artists = artists
    def closestExperts(self, genreVec):
        tempGenres = np.subtract(self.genres, genreVec)
        tempGenres = np.transpose(np.divide(np.transpose(tempGenres), self.exp))
        dists = np.sqrt([np.dot(x, x) for x in tempGenres])
        indices = np.zeros(self.k, dtype = "i")
        for i in range(self.k):
            indices[i] = np.nanargmin(dists)
            dists[indices[i]] = float('nan')
        return(indices)
    def recommend(self, genreVec):
        experts = self.closestExperts(genreVec)
        experts_artist_weight = {}
        weights = np.zeros(len(self.artists[0]))
        for i in experts:
            weights = np.add(weights, self.artists[i])
        return(weights)

In [47]:
recommender = modifiedEuclideanDistRecommender(10, userGenres, userExp, userArtists)
recommender.recommend(np.array(userGenres[2100, :]))

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


array([0., 0., 0., ..., 0., 0., 0.])